In [1]:
from netgen import gui
from ngsolve import *
from netgen.geom2d import SplineGeometry
import scipy.sparse as sp
import numpy as np

In [2]:
from netgen.geom2d import SplineGeometry
geo = SplineGeometry()
geo.AddRectangle( (0, 0), (2, 0.41), bcs = ("wall", "outlet", "wall", "inlet"))
geo.AddCircle ( (0.2, 0.2), r=0.05, leftdomain=0, rightdomain=1, bc="cyl")
mesh = Mesh( geo.GenerateMesh(maxh=0.08))
mesh.Curve(3); Draw(mesh)
# viscosity
nu = 0.001

In [3]:
k = 3
V = VectorH1(mesh,order=k, dirichlet="wall|cyl|inlet")
Q = H1(mesh,order=k-1)
X = FESpace([V,Q])

In [4]:
gfu = GridFunction(X)
velocity = gfu.components[0]
Draw(velocity,mesh,"u",sd=3)
Draw(gfu.components[1],mesh,"p",sd=3)
from ngsolve.internal import visoptions
visoptions.scalfunction = "u:0"

# parabolic inflow at bc=1:
uin = CoefficientFunction((1.5*4*y*(0.41-y)/(0.41*0.41),0))
gfu.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))

Redraw()



In [5]:
(u,p), (v,q) = X.TnT()

a = BilinearForm(X)
stokes = (nu*InnerProduct(grad(u),grad(v))-div(u)*q-div(v)*p)*dx
a += stokes
a.Assemble()

f = LinearForm(X)
f.Assemble()

inv_stokes = a.mat.Inverse(X.FreeDofs())

res = f.vec.CreateVector()
res.data = f.vec - a.mat*gfu.vec
gfu.vec.data += inv_stokes * res

Redraw()



In [6]:
dt = 0.5

In [28]:
conv = BilinearForm(X)
conv += grad(velocity)*velocity,v)*dx

In [ ]:
t = 0
tend = 0

In [ ]:
# implicit Euler/explicit Euler splitting method:
tend += 1
Mstar = BilinearForm(X)
#conv = BilinearForm(X)
while t < tend-0.5*dt:
    print ("\rt=", t, end="")
    Mstar += InnerProduct(u,v)*dx + dt*stokes + (dt*grad(velocity)*velocity*v)*dx
    Mstar.Assemble()
    conv += (dt*grad(velocity)*velocity*v)*dx
    conv.Assemble()
    
    #conv += dt*InnerProduct(grad(velocity)*velocity,v)*dx
    print(conv.mat)
    #conv.Assemble()
    #Mstar = mNS + conv.mat
    inv = Mstar.mat.Inverse(X.FreeDofs())
    
    res.data = a.mat * gfu.vec + conv.mat * gfu.vec
    gfu.vec.data -= dt * inv * res

    t = t + dt
    Redraw()